# EXAMPLE DATA

In [ ]:
# means
mn1 = 0.08
mn2 = 0.12
mn3 = 0.15

# std devs
sd1 = 0.15
sd2 = 0.25
sd3 = 0.35

# correlations
c12 = 0.15
c13 = 0.60
c23 = 0.30

# risk-free rate
rf = 0.02

# risk aversion
raver = 6

# CREATE ARRAYS

In [11]:
import numpy as np 

mn = np.array([mn1, mn2, mn3, mn4])
sd = np.array([sd1, sd2, sd3, sd4])

corr = np.identity(3)
corr[0, 1] = corr[1, 0] = c12
corr[0, 2] = corr[2, 0] = c13
corr[1, 2] = corr[2, 1] = c23

cov = np.diag(sd) @ corr @ np.diag(sd)

# CHECK CORRELATIONS

In [12]:
if np.all(np.linalg.eigvals(cov) > 0):
    print("correlations are acceptable")
else:
    print("correlations are inconsistent")

correlations are acceptable


# CALCULATE TANGENCY PORTFOLIO

In [13]:
tang = np.linalg.solve(cov, mn-rf)
tang = tang / np.sum(tang)
tang_mn = tang @ mn
tang_sd = np.sqrt(tang @ cov @ tang)

# CALCULATE OPTIMAL PORTFOLIO

In [14]:
optimal = np.linalg.solve(raver*cov, mn-rf)
optimal_mn = rf + optimal @ (mn-rf)
optimal_sd = np.sqrt(optimal @ cov @ optimal)

# CALCULATE FRONTIER

In [15]:
# global minimum variance portfolio
w = np.linalg.solve(cov, np.ones(3))
gmv = w / np.sum(w)
gmv_mn = gmv @ mn
gmv_sd = np.sqrt(gmv @ cov @ gmv)

# means to display
min_mn = 0
max_mn = 1.2*np.max(mn)
mns = np.linspace(min_mn, max_mn, 101)

# portfolio weights and risks
gmv_wt = (mns - tang_mn) / (gmv_mn - tang_mn)
tang_wt = 1 - gmv_wt
ports = (
    gmv_wt.reshape(-1, 1) * gmv.reshape(1, -1) + 
    tang_wt.reshape(-1, 1) * tang.reshape(1, -1)
)
vr = np.diag(ports @ cov @ ports.T)
sds = np.sqrt(vr)

# FIGURE

In [19]:
import plotly.graph_objects as go 
import plotly.io as pio
plotly_template = pio.templates["simple_white"]
colors = plotly_template.layout.colorway

string = 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
trace1 = go.Scatter(
    x=sds,
    y=mns,
    mode="lines",
    customdata=ports,
    hovertemplate=string
)

trace2 = go.Scatter( 
    x=sd,
    y=mn,
    text=["Asset 1", "Asset 2", "Asset 3"],
    hovertemplate="%{text}<extra></extra>",
    mode="markers",
    marker=dict(size=15)
)

string = "tangency portfolio<br>"
string += 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
trace3 = go.Scatter(
    x=[tang_sd],
    y=[tang_mn],
    customdata=tang.reshape(1, -1),
    hovertemplate=string,
    mode="markers",
    marker=dict(size=15)
)

ports = np.arange(0, 2.01, 0.01).reshape(-1, 1) * tang.reshape(1, -1)
x = np.sqrt(np.diag(ports @ cov @ ports.T))
y = rf + ports @ (mn-rf)
string = 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
trace4 = go.Scatter(
    x=x,
    y=y,
    mode="lines",
    customdata=ports,
    hovertemplate=string,
    line=dict(color=colors[5])
)

string = "optimal portfolio<br>"
string += 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
trace5 = go.Scatter(
    x=[optimal_sd],
    y=[optimal_mn],
    mode="markers",
    marker=dict(size=15),
    customdata = optimal.reshape(1, -1),
    hovertemplate=string
)

fig = go.Figure()
for trace in (trace1, trace2, trace3, trace4, trace5):
    fig.add_trace(trace)

fig.update_layout(
    xaxis_title="Standard Deviation",
    yaxis_title="Expected Return",
    xaxis_rangemode="tozero",
    yaxis_rangemode="tozero",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
    showlegend=False
) 
fig.show()

In [17]:
y

array([0.02      , 0.02080343, 0.02160686, 0.0224103 , 0.02321373,
       0.02401716, 0.02482059, 0.02562402, 0.02642746, 0.02723089,
       0.02803432, 0.02883775, 0.02964118, 0.03044462, 0.03124805,
       0.03205148, 0.03285491, 0.03365835, 0.03446178, 0.03526521,
       0.03606864, 0.03687207, 0.03767551, 0.03847894, 0.03928237,
       0.0400858 , 0.04088923, 0.04169267, 0.0424961 , 0.04329953,
       0.04410296, 0.04490639, 0.04570983, 0.04651326, 0.04731669,
       0.04812012, 0.04892355, 0.04972699, 0.05053042, 0.05133385,
       0.05213728, 0.05294071, 0.05374415, 0.05454758, 0.05535101,
       0.05615444, 0.05695788, 0.05776131, 0.05856474, 0.05936817,
       0.0601716 , 0.06097504, 0.06177847, 0.0625819 , 0.06338533,
       0.06418876, 0.0649922 , 0.06579563, 0.06659906, 0.06740249,
       0.06820592, 0.06900936, 0.06981279, 0.07061622, 0.07141965,
       0.07222308, 0.07302652, 0.07382995, 0.07463338, 0.07543681,
       0.07624024, 0.07704368, 0.07784711, 0.07865054, 0.07945